In [126]:
import pandas as pd
import numpy as np
import faiss
import time

In [17]:
#pd.set_option('display.max_rows', None)

In [48]:
#pd.reset_option('display.max_rows')

In [74]:
#d = pd.read_csv('data/tokens.txt', header=None, delimiter=' ', quoting=3)

In [77]:
#d.to_pickle('data/tokens.pkl')

In [7]:
#d = pd.read_pickle('data/tokens.pkl')

In [78]:
#used to test after reading tokens.txt to see if I had done it right
#then used to see what the first ten words were
print(d.iloc[:20, :3])

       0         1         2
0    the  0.306717 -0.320530
1      ,  0.022532  0.320788
2      . -0.075626  0.369955
3     of  0.487177 -0.193218
4     to  0.173940 -0.104660
5    and  0.222883 -0.001161
6     in  0.712315  0.451855
7      a  0.048874  0.379618
8      "  0.019833  0.136752
9      - -0.308798 -0.711716
10    's  0.777245 -0.275744
11   for  0.312680  0.062339
12  that  0.465040 -0.383865
13    on  0.285893 -0.133414
14   was  0.444401  0.233967
15    is  0.826911 -0.049384
16  said -0.434792  0.498616
17  with  0.213500  0.433140
18    he  0.753313  0.336813
19    as  0.414907 -0.157948


In [79]:
first_ten_words = d.iloc[[0,3,4,5,6,7,11,12,13,14]]

In [81]:
#cosine similarity = v1 dot v2 / (norm(v1) * norm(v2))
#cosine distance = 1 - cosine similarity

cos_similarities = np.empty((10,10))
for i in range(10):
    for j in range (10):
        cos_similarities[i, j] = np.dot(first_ten_words.iloc[i, 1:], first_ten_words.iloc[j, 1:]) / (np.linalg.norm(first_ten_words.iloc[i, 1:]) * np.linalg.norm(first_ten_words.iloc[j, 1:]))
cos_similarities = np.clip(cos_similarities, -1, 1) #clipped to fix a rounding error
cos_distances = 1 - cos_similarities

print(np.round(cos_distances, decimals=3))

[[0.    0.145 0.199 0.205 0.153 0.18  0.182 0.138 0.155 0.202]
 [0.145 0.    0.271 0.191 0.173 0.209 0.187 0.217 0.231 0.288]
 [0.199 0.271 0.    0.194 0.218 0.25  0.163 0.164 0.183 0.322]
 [0.205 0.191 0.194 0.    0.172 0.262 0.144 0.172 0.176 0.273]
 [0.153 0.173 0.218 0.172 0.    0.189 0.179 0.203 0.17  0.2  ]
 [0.18  0.209 0.25  0.262 0.189 0.    0.17  0.189 0.21  0.216]
 [0.182 0.187 0.163 0.144 0.179 0.17  0.    0.164 0.138 0.246]
 [0.138 0.217 0.164 0.172 0.203 0.189 0.164 0.    0.189 0.235]
 [0.155 0.231 0.183 0.176 0.17  0.21  0.138 0.189 0.    0.236]
 [0.202 0.288 0.322 0.273 0.2   0.216 0.246 0.235 0.236 0.   ]]


In [82]:
#angular distance = arccos(v1 dot v2 / (norm(v1) * norm(v2)))

angular_distances = np.arccos(cos_similarities)

print(np.round(angular_distances, decimals=3))

[[0.    0.545 0.642 0.652 0.56  0.61  0.613 0.532 0.564 0.647]
 [0.545 0.    0.753 0.628 0.598 0.658 0.622 0.671 0.693 0.779]
 [0.642 0.753 0.    0.633 0.673 0.723 0.579 0.581 0.614 0.826]
 [0.652 0.628 0.633 0.    0.595 0.741 0.544 0.596 0.602 0.757]
 [0.56  0.598 0.673 0.595 0.    0.626 0.607 0.649 0.592 0.644]
 [0.61  0.658 0.723 0.741 0.626 0.    0.591 0.625 0.661 0.669]
 [0.613 0.622 0.579 0.544 0.607 0.591 0.    0.581 0.531 0.717]
 [0.532 0.671 0.581 0.596 0.649 0.625 0.581 0.    0.624 0.7  ]
 [0.564 0.693 0.614 0.602 0.592 0.661 0.531 0.624 0.    0.701]
 [0.647 0.779 0.826 0.757 0.644 0.669 0.717 0.7   0.701 0.   ]]


In [83]:
ang_dis_arr_rounded_no_repeats = np.tril(np.round(angular_distances, decimals=4))
ang_dis_df = pd.DataFrame(columns=['word1', 'word2', 'angular distance'])
for i in range(10):
    for j in range(10):
        if ang_dis_arr_rounded_no_repeats[i,j] != 0:
            ang_dis_df.loc[len(ang_dis_df)] = {'word1': first_ten_words.iloc[i,0], 'word2': first_ten_words.iloc[j,0], 'angular distance': ang_dis_arr_rounded_no_repeats[i,j]}

ang_dis_dif = ang_dis_df.sort_values('angular distance')
print(ang_dis_dif)

   word1 word2  angular distance
34    on   for            0.5311
21  that   the            0.5319
18   for   and            0.5438
0     of   the            0.5455
6     in   the            0.5601
28    on   the            0.5639
17   for    to            0.5790
27  that   for            0.5813
23  that    to            0.5814
20   for     a            0.5908
32    on    in            0.5924
9     in   and            0.5945
24  that   and            0.5961
7     in    of            0.5979
31    on   and            0.6021
19   for    in            0.6070
10     a   the            0.6097
15   for   the            0.6126
30    on    to            0.6145
16   for    of            0.6223
35    on  that            0.6245
26  that     a            0.6250
14     a    in            0.6257
4    and    of            0.6276
5    and    to            0.6329
1     to   the            0.6416
40   was    in            0.6437
36   was   the            0.6470
25  that    in            0.6493
3    and  

In [ ]:
#for comparison (make sure the list is the same with both distances measures)

cos_dis_arr_rounded_no_repeats = np.tril(np.round(cos_distances, decimals=4))
cos_dis_df = pd.DataFrame(columns=['word1', 'word2', 'cosine distance'])
for i in range(10):
    for j in range(10):
        if cos_dis_arr_rounded_no_repeats[i,j] != 0:
            cos_dis_df.loc[len(cos_dis_df)] = {'word1': first_ten_words.iloc[i,0], 'word2': first_ten_words.iloc[j,0], 'cosine distance': cos_dis_arr_rounded_no_repeats[i,j]}

cos_dis_dif = cos_dis_df.sort_values('cosine distance')
print(cos_dis_dif)

In [ ]:
# QUESTION 2

In [130]:
start_time = time.time()

embeddings = d.iloc[:100000, 1:].to_numpy().astype('float32')
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

end_time = time.time()
print('Time for operation: ', end_time - start_time)

Time for operation:  0.8017599582672119


In [131]:
print('Number of vectors in index:', index.ntotal)

Number of vectors in index: 100000


In [132]:
print('Embedding array size (MB):', embeddings.nbytes / (1024*1024))

Embedding array size (MB): 38.14697265625


In [150]:
#refers to the row index from the previous cell as the query for index.search()
#the is the number of neighbors to return
query_word = 'data'
n_neighbors_to_fetch = 31

start_time = time.time()

query_word_token = embeddings[d[d[0]==query_word].index[0] : d[d[0]==query_word].index[0]+1]
distances_to_knn, indices_of_knn = index.search(query_word_token, n_neighbors_to_fetch)

end_time = time.time()
print('Time for operation: ', end_time - start_time)

Time for operation:  0.16666269302368164


In [153]:
query_word_token

array([[ 0.84648 , -0.082548, -1.301806, -0.735832,  0.674053, -0.196307,
        -1.096278, -0.580526, -0.385972, -0.283625,  0.120287, -0.952777,
         1.016617, -0.34848 , -0.342494,  0.459328,  0.695805, -0.33583 ,
         0.6236  , -0.207273,  0.335618,  1.056428,  0.557629, -0.070907,
        -0.928007, -0.65674 ,  0.343962, -0.501568, -0.754843,  0.063396,
         0.569669,  0.214937, -0.640749, -0.431692, -4.343161,  0.339861,
        -0.179838, -0.272953,  0.278215,  0.316486,  0.153532,  0.213689,
         0.29925 , -0.724164,  0.035117,  0.187556, -0.343278,  0.197821,
         0.379513,  0.502576,  0.766893,  0.476763, -0.228328,  0.294447,
        -0.714183,  0.493492,  0.930728, -0.21876 ,  0.623998, -0.252814,
        -0.233913,  0.116794, -0.119629,  0.60437 , -0.737304,  0.483543,
        -0.488996, -0.38302 , -0.735055, -0.470569, -0.019399,  0.970328,
         0.087283,  1.061838, -0.946366,  0.729165,  0.334329,  0.116828,
        -0.365285,  0.300026,  0.52113